In [6]:
import torch
import pandas as pd
import json
import os

# ==============================================================================
# [수정됨] 경로 수정: 역슬래시(\) -> 슬래시(/)
# ==============================================================================
# 파일이 현재 폴더 기준 어디에 있는지 정확해야 합니다.
# 만약 코드가 실행되는 위치가 최상위 폴더면 아래 경로가 맞습니다.
CKPT_PATH = "fp_smalltargets.pt"  
GENE_META_PATH = "gene_metadata.parquet"
OUTPUT_VOCAB_PATH = "fp_model_vocab.json" 

# 2. 체크포인트 로드
print(f"Loading checkpoint from {CKPT_PATH}...")

# 경로 존재 여부 먼저 확인 (디버깅용)
if not os.path.exists(CKPT_PATH):
    raise FileNotFoundError(f"❌ 파일을 찾을 수 없습니다: {os.path.abspath(CKPT_PATH)}")
if not os.path.exists(GENE_META_PATH):
    raise FileNotFoundError(f"❌ 파일을 찾을 수 없습니다: {os.path.abspath(GENE_META_PATH)}")

checkpoint = torch.load(CKPT_PATH, map_location="cpu")
extra = checkpoint.get("extra", {})

# 저장된 subset_token_ids 확인
subset_ids = extra.get("subset_token_ids", [])
if not subset_ids:
    raise ValueError("❌ 체크포인트에 subset_token_ids가 없습니다! 학습 코드를 확인하세요.")
print(f"✅ Found {len(subset_ids)} subset genes in checkpoint.")

# 3. 전체 유전자 메타데이터 로드
print(f"Loading gene metadata from {GENE_META_PATH}...")
gene_df = pd.read_parquet(GENE_META_PATH)

full_id_to_symbol = dict(zip(gene_df['token_id'], gene_df['gene_symbol']))

# 4. 모델 전용 Vocab 생성
special_tokens = extra.get("SPECIAL_TOKENS", ["[PAD]", "[CLS]", "[ORGAN]", "[MASK]"])
n_special = len(special_tokens)

model_vocab = {}

# (1) 스페셜 토큰 추가
for i, token in enumerate(special_tokens):
    model_vocab[token] = i

# (2) 유전자 토큰 추가
subset_genes_list = []
for i, original_token_id in enumerate(subset_ids):
    gene_symbol = full_id_to_symbol.get(original_token_id, f"UNK_{original_token_id}")
    
    model_id = n_special + i
    model_vocab[gene_symbol] = model_id
    
    subset_genes_list.append({
        "gene_symbol": gene_symbol,
        "original_token_id": original_token_id,
        "model_token_id": model_id
    })

# 5. 결과 저장
output_data = {
    "vocab_map": model_vocab,
    "subset_genes": subset_genes_list,
    "config": {
        "max_seq_len": extra.get("MAX_SEQ_LEN"),
        "d_model": extra.get("D_MODEL"),
        "vocab_size": extra.get("VOCAB_SIZE")
    }
}

with open(OUTPUT_VOCAB_PATH, "w", encoding="utf-8") as f:
    json.dump(output_data, f, indent=4, ensure_ascii=False)

print(f"✅ Successfully saved vocab to {OUTPUT_VOCAB_PATH}")
print(f"   - Total vocab size: {len(model_vocab)}")
print(f"   - Subset genes: {len(subset_genes_list)}")

Loading checkpoint from fp_smalltargets.pt...
✅ Found 4184 subset genes in checkpoint.
Loading gene metadata from gene_metadata.parquet...


C:\Users\ljm16\AppData\Local\Temp\ipykernel_35956\402263024.py:24: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(CKPT_PATH, map_location="cpu")


✅ Successfully saved vocab to fp_model_vocab.json
   - Total vocab size: 4188
   - Subset genes: 4184
